SQL IN PYTHON:

In [ ]:
import oracledb
connection = oracledb.connect(
    user="YOUR_USERNAME",
    password="YOUR_PASSWORD",
    dsn="YOUR_DSN"
)

cursor = connection.cursor()
cursor.execute("SELECT country_id, country_name FROM countries")

for row in cursor:
    print(row)

cursor.close()
connection.close()


SEARCHING WITH ENTITY:

In [ ]:
from sqlalchemy import create_engine

DB_USER = "YOUR_USERNAME"
DB_PASS = "YOUR_PASSWORD"
DB_HOST = "localhost"
DB_PORT = "1521"
DB_SERVICE = "XEPDB1"

engine = create_engine(
    f"oracle+oracledb://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/?service_name={DB_SERVICE}"
)

query = """
SELECT EMPLOYEE_ID,
       FIRST_NAME,
       LAST_NAME,
       EMAIL,
       PHONE_NUMBER
FROM HR.EMPLOYEES
WHERE FIRST_NAME LIKE '%FIND_ENTITY_HERE%'
"""

df = pd.read_sql(query, con=engine)

df.insert(0, 'Row Count', range(1, len(df) + 1))

print(df)


WEBSCRAPE:

In [ ]:
import requests
from bs4 import BeautifulSoup

TARGET_URL = "YOUR_TARGET_WEBSITE"

response = requests.get(TARGET_URL)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')

data = {}

data['Business Name'] = soup.find('title').text.strip() if soup.find('title') else None

services = soup.find_all(['h1', 'h2', 'h3'])
data['Type of Services'] = ', '.join(service.text.strip() for service in services if service)

email_tag = soup.find('a', href=lambda href: href and 'mailto:' in href)
data['Email'] = email_tag['href'].replace('mailto:', '') if email_tag else None

about_section = soup.find('div', class_='about-us') or soup.find('section', id='about')
data['Long Description'] = about_section.text.strip() if about_section else "N/A"

phone_tag = soup.find('a', href=lambda href: href and 'tel:' in href)
data['Phone #'] = phone_tag.text.strip() if phone_tag else None
data['Phone # Link'] = phone_tag['href'] if phone_tag else None

whatsapp_tag = soup.find('a', href=lambda href: href and 'wa.me' in href)
data['WhatsApp'] = whatsapp_tag['href'].split('wa.me/')[-1] if whatsapp_tag else None
data['WhatsApp # Link'] = whatsapp_tag['href'] if whatsapp_tag else None

logo_tag = soup.find('img', {'alt': 'logo'}) or soup.find('img')
data['Logo Image Link'] = logo_tag['src'] if logo_tag else None

address_section = soup.find('address') or soup.find('div', class_='address')
data['Address'] = address_section.text.strip() if address_section else "N/A"

hours_section = soup.find('div', class_='hours') or soup.find('section', id='hours')
data['Hours'] = hours_section.text.strip() if hours_section else "N/A"

facebook_tag = soup.find('a', href=lambda href: href and 'facebook.com' in href)
data['Facebook Page Link'] = facebook_tag['href'] if facebook_tag else None

instagram_tag = soup.find('a', href=lambda href: href and 'instagram.com' in href)
data['Instagram Page Link'] = instagram_tag['href'] if instagram_tag else None

for key, value in data.items():
    print(f"{key}: {value}")


WEBSCRAPING WITH GOOGLE API:

In [ ]:
import requests
import pandas as pd
import time

API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual API key
SEARCH_ENGINE_ID = "YOUR_SEARCH_ENGINE_ID"  # Replace with your actual Search Engine ID

urls_to_check = [
    "www.example.com/funding_round/sample-project", 
    "www.example.com/funding_round/another-project"
    # ... Add more as needed
]

data = []

def process_url(query, api_key, search_engine_id):
    search_url = f'https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={search_engine_id}'
    response = requests.get(search_url)
    
    data_item = {'URL': query}

    if response.status_code == 200:
        if 'items' in response.json():
            data_item['Status'] = ''  
        else:
            data_item['Status'] = 'OOS'
    elif response.status_code == 429:
        data_item['Status'] = 'Error: 429 for query'
        time.sleep(10)  
    else:
        data_item['Status'] = f"Error: {response.status_code}"

    return data_item

for query in urls_to_check:
    data_item = process_url(query, API_KEY, SEARCH_ENGINE_ID)
    data.append(data_item)

valid_urls = [item['URL'] for item in data if (item['Status'] == '' or 'Error' in item['Status']) and item['Status'] != 'OOS']

formatted_urls = '","'.join(valid_urls)

df = pd.DataFrame(data)

output_file = "websites_status_single_api_key.xlsx"
df.to_excel(output_file, index=False)
